In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_valid.shape)
print(x_test.shape,y_test.shape)

In [ ]:
# x = (x - u)/std
#u是均值，std是方差
#得到的结果符合均值是0，方差是1的正态分布

from sklearn.preprocessing import StandardScaler
sacler = StandardScaler()
#x_train:[None,28,28] ->[None,784]
#fit_transform要求数据是二维矩阵
x_train_scaled = sacler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
#归一化是所有的归一化的均值方差都要有训练集的均值方差
x_valid_scaled = sacler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaled = sacler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [ ]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

In [ ]:
#tf.keras.Sequential()
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))#将数据展平
model.add(keras.layers.Dense(300,activation="relu"))
model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))
"""
另一种表示模型的方法
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300,activation="relu"),
    keras.layers.Dense(300,activation="relu"),
    keras.layers.Dense(10,activation="softmax")
])
"""
#relu: y=max(0,x)
#softmax: 将向量变成概率分布   x=[x1,x2,x3]
#    y = [e^x1/sum,e^x2/sum,e^x3/sum]   sum = e^x1 + e^x2 + e^x3


#reason for sparse: y->index   y->one_hot->[]
#即验证集y只是一个长度为样本数目的向量，对于每个样本来说只是一个index的值，计算损失函数时需要变为向量
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [ ]:
history = model.fit(x_train,y_train,validation_data=(x_valid,y_valid),epochs=10)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()

plot_learning_curves(history)